In [ ]:
import os
import pandas as pd
from copy import copy
from datetime import datetime
from matplotlib import pyplot as plt

from autumn.projects.sm_covid2.common_school.runner_tools import INCLUDED_COUNTRIES 

from autumn.projects.sm_covid2.common_school.project_maker import get_school_project
from autumn.models.sm_covid2.stratifications.immunity import get_vacc_data
from pathlib import Path


In [ ]:
modelled_vacces = {}
vacc_datas = {}
iso3_list = list(INCLUDED_COUNTRIES["all"].keys())

In [ ]:
analysis_path = Path.home() / "Models/AuTuMN_new/user/rragonnet/remote_run_outputs/33489767_test_full_analysis_24Jan2024_main"

In [ ]:
for i, iso3 in enumerate(iso3_list):
    print(f"Running {i}-nth country ({iso3})")
    vacc_data = get_vacc_data(iso3)
    vacc_data.dropna(inplace=True)
    vacc_data['x'] = pd.to_datetime(vacc_data['date'])
    vacc_data['y'] = vacc_data['people_fully_vaccinated_per_hundred'] / 100.
    vacc_data = vacc_data.drop(["date", "people_fully_vaccinated_per_hundred"], axis=1)
    sparse_vacc_data = copy(vacc_data)
    sparse_vacc_data['keep'] = [True] * len(sparse_vacc_data)
    last_plotted_date = sparse_vacc_data.iloc[0]['x']
    for idx in sparse_vacc_data.index[1:]:
        this_date = sparse_vacc_data.loc[idx]['x']
        delta = (this_date - last_plotted_date).days
        if delta < 30:
            sparse_vacc_data['keep'].loc[idx] = False
        else:
            last_plotted_date = copy(this_date)
    sparse_vacc_data = sparse_vacc_data[sparse_vacc_data['keep'] == True]

    folder_path = analysis_path / iso3
    derived_outputs = pd.read_pickle(folder_path / "derived_outputs.pickle")
    modelled_vacc = derived_outputs["baseline"]["prop_immune_vaccinated"]

    modelled_vacces[iso3] = copy(modelled_vacc)
    vacc_datas[iso3] = copy(sparse_vacc_data)

In [ ]:
def plot_vacc(modelled_vacces, vacc_datas):
    n_rows = 11
    n_cols = 7
    plt.style.use("ggplot")
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, 20), sharex=True, sharey=True)

    i, j = 0, 0
    for iso3 in modelled_vacces:
        print(iso3)
        modelled_vacc = modelled_vacces[iso3]
        vacc_data = vacc_datas[iso3]

        ax = axs[i, j]
        modelled_vacc.plot(ax=ax)
        vacc_data.plot(x="x", y="y", ax=ax, marker=".", linewidth=0, ms=5)
        ax.set_xlim(("Jan 2021", "Dec 2022"))
        ax.set_ylim((0., 1.))
        ax.set_xlabel("")
        ax.set_ylabel("prop. vaccinated")

        ax.set_xticks(("1 Jan 2021", "1 Jan 2022"))

        ax.set_title(iso3)
        ax.get_legend().remove()

        j += 1
        if j == n_cols:
            j = 0
            i += 1

    # turn off remaing axes
    for k in range(j, n_cols):
        ax = axs[i, k]    
        ax.axis('off')

    plt.tight_layout()
    return fig

fig = plot_vacc(modelled_vacces, vacc_datas)

fig.savefig("vacc_coverage.pdf")